In [29]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

import os
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')

In [10]:
llm=ChatGroq(model='llama-3.1-8b-instant')

In [11]:
full_name=input('name')
Email_Address=input('email address')
phone_no=input('phone no')
years_of_exp=input('experience')
desired_positions=input('Desired position(s)')
current_loc=input('current location')
tech_stack=input('specify your tech stack, including programming languages, frameworks, databases, and tools you are proficient in.')

In [35]:
info='My name is {full_name}, I have {years_of_exp} years of experience, the position(s) I want to apply for is {desired_positions} and my tech stack includes {tech_stack}'
info = Document(
    page_content=info,
    metadata={'type': 'Information', 'source': 'candidate_input'}
)
doc=[info]

In [27]:
embeddings=HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

c:\Users\Chetan\OneDrive\Desktop\recruitment_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
db=FAISS.from_documents(doc,embeddings)

In [37]:
retriever=db.as_retriever()

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        ('system','You are an assistant who is specialised in interviewing and hiring candidates, you will ask every candidate 3-5 questions based on the provided tech stack, the questions should be of medium to hard type with some complexities.'),
        ('human','{input}')
    ]
)

In [13]:
parser=StrOutputParser()

In [14]:
chain=prompt | llm | parser

In [15]:
chain.invoke({'name':full_name,'input':tech_stack})

'Nice to meet you, Chetan. Based on your tech stack, I\'ll ask you some questions to assess your skills and experience. Here are 5 questions for you:\n\n1. **Python question**: You\'re working with a large dataset of user interactions in a web application. The dataset is stored in a Pandas DataFrame, and you need to perform a complex aggregation operation involving multiple join operations and grouping by a categorical column. How would you approach this problem, and what methods would you use in Python to optimize the performance of the aggregation operation?\n\n2. **Groq question**: Groq is a query language specifically designed for working with large datasets. Suppose you have a Groq query that looks like this: `SELECT * FROM table WHERE column1 = "value1" AND column2 = "value2"` . However, the `column1` and `column2` columns are not indexed, and the dataset is very large. How would you optimize this query to improve its performance, and what Groq features would you use to achieve t

In [18]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

get_history=RunnableWithMessageHistory(llm,get_session_history=get_session_history,input_messages_key='input')

In [ ]:
history_prompt=ChatPromptTemplate.from_messages(
    [
        ('system', 'You are an assistant who is specialised in interviewing and hiring candidates, you will ask every candidate 3-5 questions based on the provided tech stack, the questions should be of medium to hard type with some complexities. Do NOT answer the question, just reformulate it if needed and otherwise return it as it is.'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human','{input}')
    ]
)

In [39]:
history_retriever=create_history_aware_retriever(llm,retriever,history_prompt)

In [40]:
prompt=ChatPromptTemplate.from_messages(
    [
        ('system','You are a good assistant, give answers to the questions with respect to the context provided. keep the answers brief and should not exceed more than 3 points  \n\n context:{context}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human','{input}')
    ]
)

In [41]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(history_retriever,question_answer_chain)

In [42]:
get_history=RunnableWithMessageHistory(rag_chain,get_session_history,input_messages_key='input',history_messages_key='chat_history',output_messages_key='answer',)

In [ ]:
get_history.invoke({'input':''})